In [1]:
# initialize spark
import findspark
findspark.init()

In [2]:
from pyspark.sql.session import SparkSession
import pyspark.sql.types as tp
from pyspark.ml import Pipeline
#from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, Word2Vec
from pyspark.ml.feature import Tokenizer as pyTokenizer
#from pyspark.ml.feature import RegexTokenizer,StopWordsRemover,CountVectorizer,IDF, HashingTF
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from  pyspark.ml.pipeline import PipelineModel  # For saving the model
from pyspark.sql.functions import col
import pyspark.sql.functions as F

# Johnsnow nlp library
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

import pyspark

### Running spark

In [7]:
#!/Users/JoeKifle/spark-3.2.1-bin-hadoop3.2/bin/spark-submit --master spark://131.114.50.200:7077 kafka-script/tweet_consumer.py

In [81]:
!/Users/JoeKifle/spark-3.2.1-bin-hadoop3.2/sbin/start-master.sh

starting org.apache.spark.deploy.master.Master, logging to /Users/JoeKifle/spark-3.2.1-bin-hadoop3.2/logs/spark-JoeKifle-org.apache.spark.deploy.master.Master-1-cpe-172-100-10-56.twcny.res.rr.com.out


In [82]:
!/Users/JoeKifle/spark-3.2.1-bin-hadoop3.2/sbin/start-worker.sh spark://cpe-172-100-10-56.twcny.res.rr.com:7077

starting org.apache.spark.deploy.worker.Worker, logging to /Users/JoeKifle/spark-3.2.1-bin-hadoop3.2/logs/spark-JoeKifle-org.apache.spark.deploy.worker.Worker-1-cpe-172-100-10-56.twcny.res.rr.com.out
starting org.apache.spark.deploy.worker.Worker, logging to /Users/JoeKifle/spark-3.2.1-bin-hadoop3.2/logs/spark-JoeKifle-org.apache.spark.deploy.worker.Worker-2-cpe-172-100-10-56.twcny.res.rr.com.out
starting org.apache.spark.deploy.worker.Worker, logging to /Users/JoeKifle/spark-3.2.1-bin-hadoop3.2/logs/spark-JoeKifle-org.apache.spark.deploy.worker.Worker-3-cpe-172-100-10-56.twcny.res.rr.com.out
starting org.apache.spark.deploy.worker.Worker, logging to /Users/JoeKifle/spark-3.2.1-bin-hadoop3.2/logs/spark-JoeKifle-org.apache.spark.deploy.worker.Worker-4-cpe-172-100-10-56.twcny.res.rr.com.out


In [80]:
#!/Users/JoeKifle/spark-3.2.1-bin-hadoop3.2/sbin/stop-worker.sh

In [79]:
#!/Users/JoeKifle/spark-3.2.1-bin-hadoop3.2/sbin/stop-master.sh

### Spark Session

In [3]:
# dependency for spark-sql-kafka
conf = pyspark.SparkConf()
conf.set("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1")
conf.set("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.4.4")
#conf.set("spark.cores.max", "8")
#conf.set("spark.shuffle.service.enabled", "true")
#conf.set("spark.dynamicAllocation.enabled", "true")


#conf.set("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1")
spark_master = "spark://131.114.50.200:7077"
#spark_master = "spark://joetelila.lan:7077"
#sc = pyspark.SparkContext(master=spark_master,appName="Hello Spark")
spark = SparkSession\
        .builder\
        .master(spark_master)\
        .appName("sentimentAnalysis")\
        .config(conf=conf)\
        .getOrCreate()
#spark._sc.setLogLevel("ERROR")

22/05/13 11:28:48 WARN Utils: Your hostname, joetelila.local resolves to a loopback address: 127.0.0.1; using 192.168.1.96 instead (on interface en0)
22/05/13 11:28:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/JoeKifle/spark-3.2.1-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/JoeKifle/.ivy2/cache
The jars for the packages stored in: /Users/JoeKifle/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e1dc11d8-51d1-44e4-83ba-a871b1806b95;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.4.4 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombo

### Reading Data

In [4]:
# define the schema
my_schema = tp.StructType([
  tp.StructField(name= 'text',       dataType= tp.StringType(),   nullable= True),
  tp.StructField(name= 'polarity',    dataType= tp.IntegerType(),  nullable= True)
  ])

In [5]:
# read the dataset  
tweet_data = spark.read.csv('data/tweets_dataset_may6_no_comma.csv',inferSchema=True, header=True)

22/05/13 11:30:45 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/05/13 11:31:00 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/05/13 11:31:15 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/05/13 11:31:30 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/05/13 11:31:45 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/JoeKifle/spark-3.2

KeyboardInterrupt: 

22/05/13 11:32:00 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/05/13 11:32:15 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/05/13 11:32:30 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/05/13 11:32:45 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/05/13 11:33:00 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/05/13 11:33:15 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure th

In [ ]:
# Removing handles and links from the tweets
tweet_data = tweet_data.withColumn('text', F.regexp_replace('text','@[A-Za-z0-9_]+',''))
tweet_data = tweet_data.withColumn('text', F.regexp_replace('text','https?://[^ ]+',''))
tweet_data = tweet_data.withColumn('text', F.regexp_replace('text','www.[^ ]+',''))

In [ ]:
tweet_data.show(5)

In [ ]:
# print the schema of the file
tweet_data.printSchema()

In [ ]:
# dropping null columns
tweet_data=tweet_data.na.drop()

In [ ]:
# Show distribution of the polarity
tweet_data.groupBy("polarity") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

### 1. Logistic regression Model

In [ ]:
# Stages For the Pipeline
tokenizer = pyTokenizer(inputCol='text',outputCol='mytokens')
stopwords_remover = StopWordsRemover(inputCol='mytokens',outputCol='filtered_tokens')
word_2_vec = Word2Vec(inputCol= 'filtered_tokens', outputCol= 'w2v', vectorSize=200) #, vectorSize= 300)

In [ ]:
#model = LogisticRegression(featuresCol='vector',labelCol='polarity')
model = LogisticRegression(featuresCol= 'w2v',labelCol= 'polarity', regParam=0.008, maxIter=10000)

In [ ]:
# setup the pipeline
pipeline = Pipeline(stages= [tokenizer, stopwords_remover, word_2_vec, model])

#### Training model

In [ ]:
### Split Dataset and train
(train_tweet,test_tweet) = tweet_data.randomSplit((0.8,0.2),seed=42)
pipelineFit = pipeline.fit(train_tweet)

#### Evaluating model

In [ ]:
# Predictions on our - Test Dataset.
predictions = pipelineFit.transform(test_tweet)

evaluator = MulticlassClassificationEvaluator(labelCol='polarity',predictionCol='prediction',metricName='f1')
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)

In [ ]:
# persisting the model
pipelineFit.write().overwrite().save('pipeline_lr_model')

### 2. Logistric regression with transformation from Johnsnow nlp

#### Building pipeline

In [ ]:
#import sparknlp
#from pyspark.ml.feature import CountVectorizer, HashingTF, IDF, OneHotEncoder, StringIndexer, VectorAssembler, SQLTransformer

document_assembler = DocumentAssembler() \
      .setInputCol("text") \
      .setOutputCol("document")
tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")
normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")
stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)
stemmer = Stemmer() \
      .setInputCols(["cleanTokens"]) \
      .setOutputCol("stem")
finisher = Finisher() \
      .setInputCols(["stem"]) \
      .setOutputCols(["token_features"]) \
      .setOutputAsArray(True) \
      .setCleanAnnotations(False)
countVectors = CountVectorizer(inputCol="token_features", outputCol="features", vocabSize=10000, minDF=5)

In [ ]:
# Create the pipeline
nlp_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            stemmer, 
            finisher,
            countVectors
            ])

In [ ]:
nlp_model = nlp_pipeline.fit(tweet_data)
processed = nlp_model.transform(tweet_data)

In [ ]:
processed.select('text','token_features').show(truncate=50)

In [ ]:
processed.select('text','features','polarity').show()

#### Training the model

In [ ]:
# set seed for reproducibility
(trainingData, testData) = processed.randomSplit((0.8,0.2),seed=42)
#print("Training Dataset Count: " + str(trainingData.count()))
#print("Test Dataset Count: " + str(testData.count()))

lr = LogisticRegression(regParam=0.008, maxIter=10000, labelCol="polarity")
lrModel = lr.fit(trainingData)

#### Evaluating model

In [ ]:
predictions = lrModel.transform(testData)
# show polarity and prediction columns
predictions.select('polarity','prediction').show(20,truncate=50)

In [ ]:
# Predictions on our - Test Dataset.
#predictions = pipelineFit.transform(test_tweet)
evaluator = MulticlassClassificationEvaluator(labelCol='polarity',predictionCol='prediction',metricName='f1')
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)